In [2]:
import pandas as pd
import numpy as np
import warnings
import ast
import datetime
warnings.filterwarnings('ignore')


Cargamos las bases necesarias para el requerimiento

In [3]:
movies = pd.read_csv('movies_dataset.csv')
credits = pd.read_csv('credits.csv')

Los valores nulos de los campos revenue, budget se rellenan con el número 0.



In [4]:
movies['revenue'].fillna(0, inplace=True)
movies['budget'].fillna(0, inplace=True)

Se eliminan los duplicados de ambas bases de datos y se cambia el tipo de dato de la columna id de ambas bases y el tipo de dato Budget de la base movies. Se eliminan valores errados de movies. 

In [5]:
credits.drop_duplicates(['id'], inplace = True)
credits['id'] = credits['id'].astype(int, copy=True, errors='raise')
movies.drop_duplicates(['id'], inplace = True)
movies.drop(movies[(movies['id'] == '1997-08-20') | (movies['id'] == '2014-01-01') | (movies['id'] == '2012-09-29')].index, inplace = True)
movies['id'] = movies['id'].astype(int, copy=True, errors='raise')
movies['budget'] = movies['budget'].astype(float, copy=True, errors='raise')


Se unen las bases a través del id de la película y se eliminan las columnas innecesarias para el análisis

In [6]:
movies = movies.merge(credits, on = 'id')
movies.drop(columns = ['homepage','tagline','video','imdb_id','adult','original_title','poster_path','homepage'], inplace =  True)

Ahora desanidamos las columnas crew y cast de la nueva base de datos

In [7]:
def convert(object): # función para desanidar genero y obtener una lista con los generos así como 
    list = []        # Spoken_languaje
    for i in ast.literal_eval(object):
        list.append(i['name'])
    
    return list

In [8]:
def convert_cast(object): #función para desaninar los 5 primeros actores pricipales de las peliculas
    list = []
    counter = 0
    for i in ast.literal_eval(object):
        if counter != 5:
            list.append(i['name'])
            counter += 1
        else:
            break
    
    return list

In [9]:
def convert_direct(object): #Función para desanidar los directores de cada una de las películas
    list = []
    for i in ast.literal_eval(object):
        if i['job'] == 'Director':
            list.append(i['name'])
            break
    
    return list

Aplicamos las funciones creadas para denaidar y crear las listas correspondientes

In [10]:
movies['genres'] = movies['genres'].apply(convert)
movies['cast'] = movies['cast'].apply(convert_cast)
movies['crew'] = movies['crew'].apply(convert_direct)
movies.rename(columns={'crew':'Director'}, inplace = True)

Por último unimos los strig que tienen una separación o espacio entre palabras para mejorar la busqueda en las funciones 4 y 5 (Ejemplo: Tom Hanks - TomHanks)

In [11]:
movies ['genres'] = movies['genres'].apply(lambda x:[i.replace(' ','') for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(' ','') for i in x])
movies['Director'] = movies['Director'].apply(lambda x:[i.replace(' ','') for i in x])

Las fechas, se cambian al formato datetime AAAA-mm-dd, además se crear la columna release_year donde extraerán el año de la fecha de estreno. Y además se eliminan los valores nulos del campo release_date.



In [12]:
# Se crea la función de transformacion de strin a Datetime
def str_to_date(time):
    if len(time) < 10:
        return None
    else:
        return datetime.datetime.strptime(time,'%Y-%m-%d')

In [13]:
movies = movies.dropna(subset=['release_date'])
movies['release_date'] = movies['release_date'].apply(str_to_date)
movies['release_year'] = movies['release_date'].dt.year

Se crea la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, se retorna cero tomar el valor 0.

In [14]:
movies['return'] = round(movies['revenue'] / movies['budget'], 6) #división
movies['return'] = movies['return'].fillna(0) #retornar cero en los nulos
movies.loc[movies['return']==np.inf, 'return'] = 0 #retornar cero en los inf
movies['Director'] = movies['Director'].apply(lambda x: ' '.join(x)) #convertir a String
movies['cast'] = movies['cast'].apply(lambda x: ' '.join(x)) #convertir a String

Creación de las funciones

Funcion 1:

def cantidad_filmaciones_mes( Mes ): Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas 
en el mes consultado en la totalidad del dataset.

In [15]:
def cnt_of_films_month(Mes):
    
    months = ["enero", "febrero", "marzo", "abril", "mayo"
          , "junio", "julio", "agosto", "septiembre", 
          "octubre", "noviembre", "diciembre"]

    Mes1 = Mes.lower()
    for j in months:

        if Mes1 == j:
            c = months.index(j) + 1

    a = 0
    for mes in movies['release_date'].dt.month:
        if mes == c:
            a = a+1

    return (a) 

In [16]:
#Prueba de la función
Mes = 'Marzo'
result = cnt_of_films_month(Mes)
print(result)

3549


Función 2

def cantidad_filmaciones_dia( Dia ): Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.

In [17]:
def cnt_of_films_day(Dia):

    days = ['lunes', 'martes', 'miercoles', 'jueves', 'viernes','sabado','domingo']
    Dia1 = Dia.lower()
    for k in days:
        if k == Dia1:
            c = days.index(k)
    
    b=0
    for day in movies['release_date'].dt.weekday:
        if day == c:
            b = b+1

    return (b) 

In [18]:
#Prueba de la función
Dia = 'miercoles'
retorno = cnt_of_films_day(Dia)
print(retorno)

7027


Funcion 3:

def score_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.

In [19]:
def score_title(Titulo):
    a = 0
    Titulo = Titulo.lower()
    for film in movies['title']:
        if film.lower() == Titulo:
            x = movies[['release_date','title','popularity']].loc[movies['title'] == film]
            a += 1 
    
    print('Existen', a, 'Filmes con el título seleccionado: ')
    return x

In [20]:
#Prueba de la función
Titulo = 'Robin Hood'
filme = score_title(Titulo)
print(filme)

Existen 4 Filmes con el título seleccionado: 
      release_date       title popularity
2916    1973-11-08  Robin Hood  11.179855
12259   1922-08-18  Robin Hood   0.611926
15196   2010-05-12  Robin Hood   10.56812
45426   1991-05-13  Robin Hood   5.683753


Función 4:

def votos_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.

In [21]:
def vote_title(Titulo_filmacion):
    
    Titulo_filmacion = Titulo_filmacion.lower()
    a = 0
    for film in movies['title']:
        if film.lower() == Titulo_filmacion:
            x = movies[['title','vote_count','vote_average']].loc[movies['title'] == film]
            a = a + 1  

    if a > 1:
        if (x['vote_count'].sum() >= 2000):
            y = x['vote_count'].sum()
            z = x['vote_average'].mean()
            return f'la cantidad de votos de los {a} filmes es de {round(y)} con un promedio de votación de {z}'
        else:
            return 'No es posible hacer el calculo ya que la suma de votos es menor a 2000'
    else:
        if x['vote_count'].values[0] >= 2000:
            y = x['vote_count'].values[0]
            z = x['vote_average'].values[0]
            return f'La cantidad de votos del filme es de {round(y)} con un promedio de votación de {z}'
        else:
            return 'No es posible hacer el calculo ya que la suma de votos es menor a 2000'

In [22]:
#Prueba de la función
Titulo_filmacion = 'Robin Hood'
result1 = vote_title(Titulo_filmacion)
print(result1)

la cantidad de votos de los 4 filmes es de 2623 con un promedio de votación de 6.2


Función 5:

def get_actor( nombre_actor ): Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.

In [23]:
def get_actor(nombre_actor1):
    nombre_actor = nombre_actor1.replace(' ', '')
    actor_films = movies[movies['cast'].str.contains(nombre_actor, case=False)]['return']
    cant_movies = actor_films.count()
    return_t = actor_films.sum()
    avg_return = return_t / cant_movies
    if cant_movies > 0:
        return f"El actor {nombre_actor1} ha participado en {cant_movies} películas. Su retorno total es {round(return_t, 1)} con un promedio de {round(avg_return, 1)} por película."
    else:
        return f"No se encontraron registros para el actor {nombre_actor1}."
  

In [24]:
nombre_actor = 'Tom Hanks'
resultado = get_actor(nombre_actor)
print(resultado)

El actor Tom Hanks ha participado en 58 películas. Su retorno total es 167.8 con un promedio de 2.9 por película.


Funcion 6:
def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.


In [25]:
def get_director(nombre_director1):
    nombre_director = nombre_director1.replace(' ', '')
    films_dire = movies[movies['Director'].str.contains(nombre_director, case=False)]
    cant_movies = films_dire.shape[0]

    if cant_movies > 0:
        return_t = films_dire['return'].sum()
        dir_sucess = return_t / cant_movies
        films = films_dire[['title', 'release_date', 'return', 'budget', 'revenue']]
        films = films.reset_index(drop=True)
        print(f"El director {nombre_director} ha dirigido {cant_movies} películas. Su éxito a generado un retorno promedio de {dir_sucess:.2f} por pelicula. Las películas dirigidas son:")
        return films
    else:
        return f"No se encontraron registros para el director {nombre_director}."



In [26]:
nombre = "Forest Whitaker"
result = get_director(nombre)
print(result)

El director ForestWhitaker ha dirigido 4 películas. Su éxito a generado un retorno promedio de 1.27 por pelicula. Las películas dirigidas son:
               title release_date   return      budget     revenue
0  Waiting to Exhale   1995-12-22  5.09076  16000000.0  81452156.0
1        Hope Floats   1998-05-29  0.00000         0.0         0.0
2     First Daughter   2004-09-24  0.00000  30000000.0         0.0
3           Strapped   1993-08-21  0.00000         0.0         0.0


In [27]:
movies['spoken_languages'] = movies['spoken_languages'].apply(convert)
movies['popularity'] = movies['popularity'].astype(float)

Ya los datos están limpios, ahora es tiempo de investigar las relaciones que hay entre las variables de los datasets, ver si hay outliers o anomalías (que no tienen que ser errores necesariamente 👀 ), y ver si hay algún patrón interesante que valga la pena explorar en un análisis posterior. Las nubes de palabras dan una buena idea de cuáles palabras son más frecuentes en los títulos, ¡podría ayudar al sistema de recomendación! Sabes que puedes apoyarte en librerías como pandas profiling, missingno, sweetviz, autoviz, entre otros y sacar de allí tus conclusiones 

In [28]:
movies['genres'] = movies['genres'].apply(lambda x: ' '.join(x)) 
movies['tags'] =  movies['overview'] + movies['genres'] + ' '+ movies['cast'] + movies['Director']
movies_new = movies[['id', 'title', 'tags']]
movies_new = movies_new.dropna(subset=['tags'])

In [29]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()


In [30]:
def stem(text):
    list1 = []
    for i in text.split():
        list1.append(ps.stem(i))
    
    return " ".join(list1)

In [31]:
movies_new['tags'] = movies_new['tags'].apply(stem)
movies_p = movies_new.head(20000) #selecciono una muestra de 20000

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features= 9000, stop_words='english')
vector = cv.fit_transform(movies_p['tags']).toarray()
cv.get_feature_names_out()

array(['000', '10', '100', ..., 'zoo', 'zooeydeschanel', 'моррисчестнат'],
      dtype=object)

In [33]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [39]:
def movie_recommend(movie):
    movie_index = movies_p[movies_p['title']  == movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)),reverse = True, key = lambda x:x[1])[1:6]
    listM = []
    for i in movie_list:
        
        j = movies_p.iloc[i[0]].title
        listM.append(j)
    
    return listM


In [41]:
movie_recommend("Toy Story")

['Toy Story 2',
 'Toy Story 3',
 'Walk Like a Man',
 'Happiness Is a Warm Blanket, Charlie Brown',
 'Carried Away']